## The imports

In [ ]:
import zarr
import xarray as xr
import time
import dask
from intake import open_catalog


# The workers

In [ ]:
nb_workers=40

In [ ]:
from dask.distributed import Client
from dask_gateway import Gateway

gateway = Gateway()
cluster = gateway.new_cluster()
cluster.scale(nb_workers)
c = Client(cluster)
c

In [ ]:
from dask.utils import ensure_dict, format_bytes
workers = c.scheduler_info()["workers"]
text="Workers= " + str(len(workers))
memory = [w["memory_limit"] for w in workers.values()]
if all(memory):
    text += ", Memory=" + format_bytes(sum(memory))
print(text)


# The data

In [ ]:
%%time 
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/MEOM-NEMO.yaml")
ds  = cat["eNATL60_BLBT02_SSH"].to_dask()

In [ ]:
ds

In [ ]:
%time mean=ds.sossheig.mean(dim='time_counter')

In [ ]:
mean

In [ ]:
%time mean.load()

In [ ]:
%time mean_space=ds.sossheig.mean(dim='x').mean(dim='y')

In [ ]:
mean_space

In [ ]:
%time mean_space.load()

In [ ]:
cluster.close()